**Dealing with Missing Values**

One of the first steps in building a good predictive model is to carefully handle missing values at the start. 
There's quite a lot of missing data in this dataset.

After investigating the data in some detail there also appears to be some fields which represent similar if not the same information which I think we can probably be remove as they are redundant. 

There are also potentially some inconsistent fields and potentially incorrect data.

The approaches use to deal with missing values also have to be made to the test data consistently.

Ideally after each step taken to deal with missing values you would probably want to carry out some cross-validation to see if it has helped improve your model.

**Feature Engineer**:

transaction_mon = month(transactiondate)

lati_cd = input(substr(put(latitude, 8.), 1, 4), 3.0)

**Replace Missing with most Frequent Number**:

if buildingqualitytypeid = . then buildingqualitytypeid = 8;

if heatingorsystemtypeid = . then heating orsystemtypeid = 2;

**Replace Missing with Avg between 25 to 75 tax value Quantile**:

if lotsizesquarefeet = . then lotsizesquarefeet = 32065;

if unitcnt = . then unitcnt = 1;

In [1]:
from boto.s3.connection import S3Connection
import os
import json
import boto.s3
import sys
import datetime
import seaborn as sns
from boto.s3.key import Key
from pprint import pprint
import pandas as pd
import urllib
import csv
import io
import requests
import time
import json
import datetime
from pprint import pprint
import scipy
import numpy as np
import matplotlib.pyplot as plt
color = sns.color_palette()

%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import neighbors
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

In [2]:
rawdata= pd.read_csv("C:/Users/Deepika Keswarap/Documents/projecta/all//train_2017.csv") 
rawdata.shape

(77613, 3)

In [3]:
prop_df = pd.read_csv("C:/Users/Deepika Keswarap/Documents/projecta/all/properties_2017.csv")
prop_df.shape

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(2985217, 58)

In [4]:
for c, dtype in zip(prop_df.columns, prop_df.dtypes):
    if dtype == np.float64:
        prop_df[c] = prop_df[c].astype(np.float32)

df_train = rawdata.merge(prop_df, how='left', on='parcelid')
#del prop_df, rawdataspecificrows
df_train.head(2)
#df_train.shape

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,14297519,0.025595,2017-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,NaN,33634932.0,-117869208.0,4506.0,NaN,NaN,NaN,NaN,NaN,122,261.0,NaN,60590632.0,53571.0,1286.0,NaN,96978.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.719727,NaN,NaN,6.059063e+13
1,17052889,0.055619,2017-01-01,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,NaN,34449264.0,-119281528.0,12647.0,NaN,NaN,NaN,NaN,NaN,1110,261.0,NaN,61110012.0,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.479980,NaN,NaN,6.111001e+13


In [5]:
prop_df.dtypes



parcelid                          int64
airconditioningtypeid           float32
architecturalstyletypeid        float32
basementsqft                    float32
bathroomcnt                     float32
bedroomcnt                      float32
buildingclasstypeid             float32
buildingqualitytypeid           float32
calculatedbathnbr               float32
decktypeid                      float32
finishedfloor1squarefeet        float32
calculatedfinishedsquarefeet    float32
finishedsquarefeet12            float32
finishedsquarefeet13            float32
finishedsquarefeet15            float32
finishedsquarefeet50            float32
finishedsquarefeet6             float32
fips                            float32
fireplacecnt                    float32
fullbathcnt                     float32
garagecarcnt                    float32
garagetotalsqft                 float32
hashottuborspa                   object
heatingorsystemtypeid           float32
latitude                        float32


#### Pools & Hot tubs

There are actually multiple features related to pools:

- **"poolcnt"** - Number of pools on a lot. "NaN" means "0 pools", so we can update that to reflect "0" instead of "NaN".


- **"hashottuborspa"** - Does the home have a hottub or a spa? "NaN" means "0 hottubs or spas", so we can update that to reflect "0" instead of "NaN".


- **"poolsizesum"** - Total square footage of pools on property. Similarly, "NaN" means "0 sqare feet of pools", so we can also adjust that to read "0". For homes that do have pools, but are missing this information, we will just fill the "NaN" with the median value of other homes with pools.


- **"pooltypeid2"** & **"pooltypeid7"** & **"pooltypeid10"** - Type of pool or hottub present on property. These categories will only contain non-null information if "poolcnt" or "hashottuborspa" contain non-null information. For the pool-related categories, we can fill the "NaN" value with a "0". And because "pooltypeid10" tells us the exact same information as "hashottuborspa", we can probably drop that category from our model.

In [6]:
# "0 pools"
df_train.poolcnt.fillna(0,inplace = True)
df_train.head(2)

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,14297519,0.025595,2017-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,NaN,33634932.0,-117869208.0,4506.0,0.0,NaN,NaN,NaN,NaN,122,261.0,NaN,60590632.0,53571.0,1286.0,NaN,96978.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.719727,NaN,NaN,6.059063e+13
1,17052889,0.055619,2017-01-01,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,NaN,34449264.0,-119281528.0,12647.0,0.0,NaN,NaN,NaN,NaN,1110,261.0,NaN,61110012.0,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.479980,NaN,NaN,6.111001e+13


In [7]:
# "0 hot tubs or spas"
df_train.hashottuborspa.fillna(0, inplace = True)
df_train.head(2)

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,14297519,0.025595,2017-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,0,NaN,33634932.0,-117869208.0,4506.0,0.0,NaN,NaN,NaN,NaN,122,261.0,NaN,60590632.0,53571.0,1286.0,NaN,96978.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.719727,NaN,NaN,6.059063e+13
1,17052889,0.055619,2017-01-01,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,0,NaN,34449264.0,-119281528.0,12647.0,0.0,NaN,NaN,NaN,NaN,1110,261.0,NaN,61110012.0,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.479980,NaN,NaN,6.111001e+13


In [8]:
# Convert "True" to 1
df_train.hashottuborspa.replace(to_replace = True, value = 1,inplace = True)
df_train.head(2)

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,14297519,0.025595,2017-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,0,NaN,33634932.0,-117869208.0,4506.0,0.0,NaN,NaN,NaN,NaN,122,261.0,NaN,60590632.0,53571.0,1286.0,NaN,96978.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.719727,NaN,NaN,6.059063e+13
1,17052889,0.055619,2017-01-01,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,0,NaN,34449264.0,-119281528.0,12647.0,0.0,NaN,NaN,NaN,NaN,1110,261.0,NaN,61110012.0,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.479980,NaN,NaN,6.111001e+13


In [9]:
print(df_train['hashottuborspa'].value_counts())

0    76074
1     1539
Name: hashottuborspa, dtype: int64


In [10]:
# Set properties that have a pool but no info on poolsize equal to the median poolsize value.
df_train.loc[df_train.poolcnt==1, 'poolsizesum'] = df_train.loc[df_train.poolcnt==1, 'poolsizesum'].fillna(df_train[df_train.poolcnt==1].poolsizesum.median())

# "0 pools" = "0 sq ft of pools"
df_train.loc[df_train.poolcnt==0, 'poolsizesum']=0

# "0 pools with a spa/hot tub"
df_train.pooltypeid2.fillna(0,inplace = True)

# "0 pools without a hot tub"
df_train.pooltypeid7.fillna(0,inplace = True)

# Drop redundant feature
df_train.drop('pooltypeid10', axis=1, inplace=True)

df_train.head(2)

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,14297519,0.025595,2017-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,0,NaN,33634932.0,-117869208.0,4506.0,0.0,0.0,0.0,0.0,122,261.0,NaN,60590632.0,53571.0,1286.0,NaN,96978.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.719727,NaN,NaN,6.059063e+13
1,17052889,0.055619,2017-01-01,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,0,NaN,34449264.0,-119281528.0,12647.0,0.0,0.0,0.0,0.0,1110,261.0,NaN,61110012.0,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.479980,NaN,NaN,6.111001e+13


#### Fireplace Data

There are two features related to fireplaces:

- **"fireplaceflag"** - Does the home have a fireplace? The answers are either "True" or "NaN". We will change the "True" values to "1" and the "NaN" values to "0".

- **"fireplacecnt"** - How many fireplaces in the home? We can replace "NaN" values with "0".

Looking deeper, it seems odd that over 10% of the homes have 1 or more fireplaces according to the "fireplacecnt" feature, but less than 1% of homes actually have "fireplaceflag" set to "True". There are obviously some errors with this data collection. To fix this, we will do the following:

- If "fireplaceflag" is "True" and "fireplacecnt" is "NaN", we will set "fireplacecnt" equal to the median value of "1".

- If "fireplacecnt" is 1 or larger "fireplaceflag" is "NaN", we will set "fireplaceflag" to "True".

We will change "True" in "fireplaceflag" to "1", so we can more easily analyze the information.

In [11]:
print(df_train['fireplaceflag'].value_counts())

True    172
Name: fireplaceflag, dtype: int64


In [12]:
# If "fireplaceflag" is "True" and "fireplacecnt" is "NaN", we will set "fireplacecnt" equal to the median value of "1".
df_train.loc[(df_train['fireplaceflag'] == True) & (df_train['fireplacecnt'].isnull()), ['fireplacecnt']] = 1

In [13]:
# If 'fireplacecnt' is "NaN", replace with "0"
df_train.fireplacecnt.fillna(0,inplace = True)

In [14]:
# If "fireplacecnt" is 1 or larger "fireplaceflag" is "NaN", we will set "fireplaceflag" to "True".
df_train.loc[(df_train['fireplacecnt'] >= 1.0) & (df_train['fireplaceflag'].isnull()), ['fireplaceflag']] = True
df_train.fireplaceflag.fillna(0,inplace = True)

In [15]:
print(df_train['fireplaceflag'].value_counts())

0       69152
True     8461
Name: fireplaceflag, dtype: int64


In [16]:
# Convert "True" to 1
df_train.fireplaceflag.replace(to_replace = True, value = 1,inplace = True)

In [17]:
df_train.head(2)

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,14297519,0.025595,2017-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,0.0,3.0,2.0,633.0,0,NaN,33634932.0,-117869208.0,4506.0,0.0,0.0,0.0,0.0,122,261.0,NaN,60590632.0,53571.0,1286.0,NaN,96978.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,1998.0,NaN,0,485713.0,1023282.0,2016.0,537569.0,11013.719727,NaN,NaN,6.059063e+13
1,17052889,0.055619,2017-01-01,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,0,NaN,34449264.0,-119281528.0,12647.0,0.0,0.0,0.0,0.0,1110,261.0,NaN,61110012.0,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,1.0,1,88000.0,464000.0,2016.0,376000.0,5672.479980,NaN,NaN,6.111001e+13


#### Garage Data

There are two features related to garages:

- **"garagecarcnt"** - How many garages does the house have? Easy fix here - we can replace "NaN" with "0" if a house doesn't have a garage.


- **"garagetotalsqft"** - What is the square footage of the garage? Again, if a home doesn't have a garage, we can replace "NaN" with "0".

Unlike the Fireplace category where we have several Type II errors (false negative), we do not have any scenarios where a home has a "garagecarcnt" of "NaN", but a "garagetotalsqft" of some value.

In [18]:
df_train.garagecarcnt.fillna(0,inplace = True)
df_train.garagetotalsqft.fillna(0,inplace = True)

In [19]:
df_train.head(2)

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,14297519,0.025595,2017-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,0.0,3.0,2.0,633.0,0,NaN,33634932.0,-117869208.0,4506.0,0.0,0.0,0.0,0.0,122,261.0,NaN,60590632.0,53571.0,1286.0,NaN,96978.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,1998.0,NaN,0,485713.0,1023282.0,2016.0,537569.0,11013.719727,NaN,NaN,6.059063e+13
1,17052889,0.055619,2017-01-01,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,0,NaN,34449264.0,-119281528.0,12647.0,0.0,0.0,0.0,0.0,1110,261.0,NaN,61110012.0,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,1.0,1,88000.0,464000.0,2016.0,376000.0,5672.479980,NaN,NaN,6.111001e+13


#### Tax Data Delinquency

There are two features related to tax delinquency:

- **"taxdelinquencyflag"** - Property taxes for this parcel are past due as of 2015.


- **"taxdelinquencyyear"** - Year for which the unpaid property taxes were due.

In [20]:
print(df_train['taxdelinquencyflag'].value_counts())

Y    2900
Name: taxdelinquencyflag, dtype: int64


In [21]:
# Replace "NaN" with "0"
df_train.taxdelinquencyflag.fillna(0,inplace = True)

# Change "Y" to "1"
df_train.taxdelinquencyflag.replace(to_replace = 'Y', value = 1,inplace = True)

In [22]:
# Drop "taxdelinquencyyear" because it probably isn't a necessary variable
df_train.drop('taxdelinquencyyear', axis=1, inplace=True)

In [23]:
df_train.head(2)

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,censustractandblock
0,14297519,0.025595,2017-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,0.0,3.0,2.0,633.0,0,NaN,33634932.0,-117869208.0,4506.0,0.0,0.0,0.0,0.0,122,261.0,NaN,60590632.0,53571.0,1286.0,NaN,96978.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,1998.0,NaN,0,485713.0,1023282.0,2016.0,537569.0,11013.719727,0,6.059063e+13
1,17052889,0.055619,2017-01-01,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,0,NaN,34449264.0,-119281528.0,12647.0,0.0,0.0,0.0,0.0,1110,261.0,NaN,61110012.0,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,1.0,1,88000.0,464000.0,2016.0,376000.0,5672.479980,0,6.111001e+13


#### The Rest

- **"storytypeid"** - Numerical ID that describes all types of homes. Mostly missing, so we should drop this category. Crazy idea would be to try and integrate street view of each home, and use image recognition to classify each type of story ID.

In [24]:
# Drop "storytypeid"
df_train.drop('storytypeid', axis=1, inplace=True)

- **"basementsqft"** - Square footage of basement. Mostly missing, suggesting no basement, so we will replace "NaN" with "0".

In [25]:
# Replace "NaN" with 0, signifying no basement.
df_train.basementsqft.fillna(0,inplace = True)

- **"yardbuildingsqft26"** - Storage shed square footage. We can set "NaN" values to "0". Might be useful to change this to a categorical category of just "1"s and "0"s (has a shed vs doesn't have a storage shed), but some of the sheds are enormous and others are tiny, so we will keep the actual square footage.

In [26]:
# Replace 'yardbuildingsqft26' "NaN"s with "0".
df_train.yardbuildingsqft26.fillna(0,inplace = True)

- **"architecturalstyletypeid"** - What is the architectural style of the house? Examples: ranch, bungalow, Cape Cod, etc. Because this is only present in a small fraction of the homes, I'm going to drop this category. (Idea: One can also assume that most homes in the same neighborhood have the same style. Could also try image recognition.)

In [27]:
# Drop "architecturalstyletypeid"
df_train.drop('architecturalstyletypeid', axis=1, inplace=True)

- **"typeconstructiontypeid"** - What material is the house made out of? Missing in a bunch, so probably drop category. Would be very difficult image recognition problem.

- **"finishedsquarefeet13"** - Perimeter of living area. This seems more like describing the shape of the house and is closely related to the square footage. I recommend dropping the category.

In [28]:
# Drop "typeconstructiontypeid" and "finishedsquarefeet13"
df_train.drop('typeconstructiontypeid', axis=1, inplace=True)
df_train.drop('finishedsquarefeet13', axis=1, inplace=True)

- **"buildingclasstypeid"** - Describes the internal structure of the home. Not a lot of information gained and present in less than 1% of properties. I will drop.

In [29]:
# Drop "buildingclasstypeid"
df_train.drop('buildingclasstypeid', axis=1, inplace=True)

In [30]:
print(df_train.shape)
df_train.notnull().mean().sort_values(ascending = False)

(77613, 53)


poolcnt                         1.000000
pooltypeid7                     1.000000
logerror                        1.000000
transactiondate                 1.000000
basementsqft                    1.000000
fireplacecnt                    1.000000
garagecarcnt                    1.000000
garagetotalsqft                 1.000000
hashottuborspa                  1.000000
taxdelinquencyflag              1.000000
poolsizesum                     1.000000
pooltypeid2                     1.000000
parcelid                        1.000000
fireplaceflag                   1.000000
yardbuildingsqft26              1.000000
propertycountylandusecode       0.999562
latitude                        0.999562
longitude                       0.999562
fips                            0.999562
bedroomcnt                      0.999562
bathroomcnt                     0.999562
roomcnt                         0.999562
propertylandusetypeid           0.999562
assessmentyear                  0.999562
rawcensustractan

- **"decktypeid"** - Type of deck (if any) on property. Looks like a value is either "66.0" or "NaN". I will keep this feature and change the "66.0" to "1" for "Yes" and "NaN" to "0" for "No".

In [31]:
# Let's check the unique values for "decktypeid"
print(df_train['decktypeid'].value_counts())

66.0    614
Name: decktypeid, dtype: int64


In [32]:
# Change "decktypeid" "Nan"s to "0"
df_train.decktypeid.fillna(0,inplace = True)
# Convert "decktypeid" "66.0" to "1"
df_train.decktypeid.replace(to_replace = 66.0, value = 1,inplace = True)

In [33]:
print(df_train['decktypeid'].value_counts())
df_train.head(2)

0.0    76999
1.0      614
Name: decktypeid, dtype: int64


,parcelid,logerror,transactiondate,airconditioningtypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,censustractandblock
0,14297519,0.025595,2017-01-01,NaN,0.0,3.5,4.0,NaN,3.5,0.0,NaN,3100.0,3100.0,NaN,NaN,NaN,6059.0,0.0,3.0,2.0,633.0,0,NaN,33634932.0,-117869208.0,4506.0,0.0,0.0,0.0,0.0,122,261.0,NaN,60590632.0,53571.0,1286.0,NaN,96978.0,0.0,1.0,NaN,NaN,0.0,1998.0,NaN,0,485713.0,1023282.0,2016.0,537569.0,11013.719727,0,6.059063e+13
1,17052889,0.055619,2017-01-01,NaN,0.0,1.0,2.0,NaN,1.0,0.0,1465.0,1465.0,1465.0,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,0,NaN,34449264.0,-119281528.0,12647.0,0.0,0.0,0.0,0.0,1110,261.0,NaN,61110012.0,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,0.0,1967.0,1.0,1,88000.0,464000.0,2016.0,376000.0,5672.479980,0,6.111001e+13


- **"finishedsquarefeet6"** - Base unfinished and finished area. Not sure what this means. Seems like it gives valuable information, but replacing "NaN"s with "0"s would be incorrect. Perhaps it is a subset of other categories. Probably drop, but TBD.

- **"finishedsquarefeet15"** - Total area. Should be equal to sum of all other finishedsquarefeet categories.

- **"finishedfloor1squarefeet"** - Sq footage of first floor. Could cross check this with number of stories.

- **"finishedsquarefeet50"** - Identical to above category? Drop one of them. Duplicate.

- **"finishedsquarefeet12"** - Finished living area.

- **"calculatedfinishedsquarefeet"** - Total finished living area of home.

In [34]:
squarefeet = df_train[df_train['finishedsquarefeet15'].notnull() & 
                      df_train['finishedsquarefeet50'].notnull() & 
                      df_train['lotsizesquarefeet'].notnull()]
squarefeet[['calculatedfinishedsquarefeet','finishedsquarefeet6','finishedsquarefeet12','finishedsquarefeet15','finishedsquarefeet50','numberofstories','lotsizesquarefeet','landtaxvaluedollarcnt','structuretaxvaluedollarcnt','taxvaluedollarcnt','taxamount']]

,calculatedfinishedsquarefeet,finishedsquarefeet6,finishedsquarefeet12,finishedsquarefeet15,finishedsquarefeet50,numberofstories,lotsizesquarefeet,landtaxvaluedollarcnt,structuretaxvaluedollarcnt,taxvaluedollarcnt,taxamount
40433,928.0,NaN,NaN,928.0,928.0,1.0,6296.0,306434.0,104411.0,410845.0,4401.859863
54978,1500.0,NaN,NaN,1500.0,1500.0,1.0,24375.0,182039.0,104022.0,286061.0,4477.459961


**"finishedsquarefeet6"** is rarely present, and even when it is present, it is equal to **"calculatedfinishedsquarefeet"**. Because of this, we will drop it. Same scenario with **"finishedsquarefeet12"**, so we will drop that as well. **"finishedsquarefeet50"** is identical to **"finishedfloor1squarefeet"**, so we will also drop **"finishedfloor1squarefeet"**.

In [35]:
# Drop "finishedsquarefeet6"
df_train.drop('finishedsquarefeet6', axis=1, inplace=True)

# Drop "finishedsquarefeet12"
df_train.drop('finishedsquarefeet12', axis=1, inplace=True)

# Drop "finishedfloor1squarefeet"
df_train.drop('finishedfloor1squarefeet', axis=1, inplace=True)

df_train.head(2)

,parcelid,logerror,transactiondate,airconditioningtypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,calculatedfinishedsquarefeet,finishedsquarefeet15,finishedsquarefeet50,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,censustractandblock
0,14297519,0.025595,2017-01-01,NaN,0.0,3.5,4.0,NaN,3.5,0.0,3100.0,NaN,NaN,6059.0,0.0,3.0,2.0,633.0,0,NaN,33634932.0,-117869208.0,4506.0,0.0,0.0,0.0,0.0,122,261.0,NaN,60590632.0,53571.0,1286.0,NaN,96978.0,0.0,1.0,NaN,NaN,0.0,1998.0,NaN,0,485713.0,1023282.0,2016.0,537569.0,11013.719727,0,6.059063e+13
1,17052889,0.055619,2017-01-01,NaN,0.0,1.0,2.0,NaN,1.0,0.0,1465.0,NaN,1465.0,6111.0,1.0,1.0,1.0,0.0,0,NaN,34449264.0,-119281528.0,12647.0,0.0,0.0,0.0,0.0,1110,261.0,NaN,61110012.0,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,0.0,1967.0,1.0,1,88000.0,464000.0,2016.0,376000.0,5672.479980,0,6.111001e+13


In [36]:
squarefeet2 = df_train[df_train['finishedsquarefeet15'].notnull() & 
                            df_train['finishedsquarefeet50'].notnull() & 
                            df_train['lotsizesquarefeet'].notnull()]

squarefeet2[['calculatedfinishedsquarefeet','finishedsquarefeet15','finishedsquarefeet50','numberofstories','lotsizesquarefeet']]

,calculatedfinishedsquarefeet,finishedsquarefeet15,finishedsquarefeet50,numberofstories,lotsizesquarefeet
40433,928.0,928.0,928.0,1.0,6296.0
54978,1500.0,1500.0,1500.0,1.0,24375.0


In [37]:
df_train.notnull().mean().sort_values(ascending = False)

poolsizesum                     1.000000
pooltypeid7                     1.000000
logerror                        1.000000
transactiondate                 1.000000
basementsqft                    1.000000
decktypeid                      1.000000
fireplacecnt                    1.000000
garagecarcnt                    1.000000
garagetotalsqft                 1.000000
hashottuborspa                  1.000000
poolcnt                         1.000000
taxdelinquencyflag              1.000000
pooltypeid2                     1.000000
parcelid                        1.000000
yardbuildingsqft26              1.000000
fireplaceflag                   1.000000
roomcnt                         0.999562
propertycountylandusecode       0.999562
fips                            0.999562
latitude                        0.999562
longitude                       0.999562
bedroomcnt                      0.999562
bathroomcnt                     0.999562
propertylandusetypeid           0.999562
assessmentyear  

- **"calculatedfinishedsquarefeet"** - Present in 98%. Total finished living area of home. Let's fill the rest with the median values.

- **"finishedsquarefeet15"** - Present in 6.4%. Most cases, it is equal to **"calculatedfinishedsquarefeet"**, so we will fill in the "NaN" values with the value of "calculatedfinishedsquarefeet". Total area. Should be equal to sum of all other finishedsquarefeet categories.

- **"finishedsquarefeet50"** - If **"numberofstories"** is equal to "1", then we can replace the "NaN"s with the **"calculatedfinishedsquarefeet"** value. Fill in the rest with the average values.

In [38]:
# Replace "NaN" "calculatedfinishedsquarefeet" values with mean.
df_train['calculatedfinishedsquarefeet'].fillna((df_train['calculatedfinishedsquarefeet'].mean()), inplace=True)

In [39]:
# Replace "NaN" "finishedsquarefeet15" values with calculatedfinishedsquarefeet.
df_train.loc[df_train['finishedsquarefeet15'].isnull(),'finishedsquarefeet15'] = df_train['calculatedfinishedsquarefeet']

In [40]:
df_train.numberofstories.fillna(1,inplace = True)

In [41]:
print(df_train['numberofstories'].value_counts())

1.0    70411
2.0     6764
3.0      437
6.0        1
Name: numberofstories, dtype: int64


In [42]:
# If "numberofstories" is equal to "1", then we can replace the "NaN"s with the "calculatedfinishedsquarefeet" value. Fill in the rest with the average values.
df_train.loc[df_train['numberofstories'] == 1.0,'finishedsquarefeet50'] = df_train['calculatedfinishedsquarefeet']
df_train['finishedsquarefeet50'].fillna((df_train['finishedsquarefeet50'].mean()), inplace=True)

In [43]:
print(df_train.shape)
df_train.notnull().mean().sort_values(ascending = False)

(77613, 50)


poolsizesum                     1.000000
pooltypeid7                     1.000000
logerror                        1.000000
transactiondate                 1.000000
basementsqft                    1.000000
decktypeid                      1.000000
calculatedfinishedsquarefeet    1.000000
finishedsquarefeet15            1.000000
finishedsquarefeet50            1.000000
fireplacecnt                    1.000000
garagecarcnt                    1.000000
garagetotalsqft                 1.000000
hashottuborspa                  1.000000
poolcnt                         1.000000
taxdelinquencyflag              1.000000
pooltypeid2                     1.000000
parcelid                        1.000000
yardbuildingsqft26              1.000000
fireplaceflag                   1.000000
numberofstories                 1.000000
longitude                       0.999562
assessmentyear                  0.999562
bathroomcnt                     0.999562
bedroomcnt                      0.999562
fips            

- **"yardbuildingsqft17"** - Patio in yard. Do same as storage shed category.

In [44]:
# Replace 'yardbuildingsqft17' "NaN"s with "0".
df_train.yardbuildingsqft17.fillna(0,inplace = True)

Now let's dig into the bathroom features.

- **"threequarterbathnbr"** - Number of 3/4 baths = shower, sink, toilet.

- **"fullbathcnt"** - Number of full bathrooms - tub, sink, toilet

- **"calculatedbathnbr"** - Total number of bathrooms including partials.

It seems like **"calculatedbathnbr"** should encompass the other two, so I will probably drop **"threequarterbathnbr"** and **"fullbathcnt"**, but let's take a look at some data first...

In [45]:
bathrooms = df_train[df_train['fullbathcnt'].notnull() & 
                     df_train['threequarterbathnbr'].notnull() & 
                     df_train['calculatedbathnbr'].notnull()]
bathrooms[['fullbathcnt','threequarterbathnbr','calculatedbathnbr']]

,fullbathcnt,threequarterbathnbr,calculatedbathnbr
0,3.0,1.0,3.5
12,2.0,1.0,2.5
13,2.0,1.0,2.5
15,2.0,1.0,2.5
21,2.0,1.0,2.5
22,2.0,1.0,2.5
26,4.0,1.0,4.5
31,2.0,1.0,2.5
36,2.0,1.0,2.5
39,2.0,1.0,2.5


It looks like **"threequarterbathnbr"** is only a half-bath. Because **"calculatedbathnbr"** incorporates the other two, we will drop them. Then we will fill in the missing values for **"calculatedbathnbr"** with the most common answer.

In [46]:
# Drop "threequarterbathnbr"
df_train.drop('threequarterbathnbr', axis=1, inplace=True)

# Drop "fullbathcnt"
df_train.drop('fullbathcnt', axis=1, inplace=True)

# Fill in "NaN" "calculatedbathnbr" with most common
bathroommode = df_train['calculatedbathnbr'].value_counts().idxmax()
df_train['calculatedbathnbr'] = df_train['calculatedbathnbr'].fillna(bathroommode)


In [47]:
print(df_train.shape)
df_train.notnull().mean().sort_values(ascending = False)

(77613, 48)


poolsizesum                     1.000000
finishedsquarefeet50            1.000000
taxdelinquencyflag              1.000000
poolcnt                         1.000000
hashottuborspa                  1.000000
garagetotalsqft                 1.000000
garagecarcnt                    1.000000
fireplacecnt                    1.000000
finishedsquarefeet15            1.000000
pooltypeid7                     1.000000
calculatedfinishedsquarefeet    1.000000
decktypeid                      1.000000
calculatedbathnbr               1.000000
basementsqft                    1.000000
transactiondate                 1.000000
logerror                        1.000000
pooltypeid2                     1.000000
parcelid                        1.000000
yardbuildingsqft26              1.000000
fireplaceflag                   1.000000
numberofstories                 1.000000
yardbuildingsqft17              1.000000
latitude                        0.999562
assessmentyear                  0.999562
bathroomcnt     

- **"airconditioningtypeid"** - If "NaN", change to "5" for "None".

In [48]:
df_train.airconditioningtypeid.fillna(5,inplace = True)

- **"regionidneighborhood"** - Neighborhood. Could fill in blanks. Would need a key that maps lat & longitude regions with specific neighborhoods. Because **"longitude"** and **"latitude"** essentially provide this information, we will drop **"regionidneighborhood"**.

In [49]:
# Drop "regionidneighborhood"
df_train.drop('regionidneighborhood', axis=1, inplace=True)

- **"heatingorsystemtypeid"** - Change "NaN" to "13" for "None" *REVIST THIS*

In [50]:
df_train.heatingorsystemtypeid.fillna(13,inplace = True)

- **"buildingqualitytypeid"** - Change "NaN" to most common value.

In [51]:
print(df_train['buildingqualitytypeid'].value_counts())

8.0     14685
6.0     13501
4.0      9508
7.0      5981
9.0      2303
5.0      2099
11.0      933
3.0       335
10.0      313
12.0      116
1.0        26
2.0         9
Name: buildingqualitytypeid, dtype: int64


In [52]:
# Fill in "NaN" "buildingqualitytypeid" with most common
buildingqual = df_train['buildingqualitytypeid'].value_counts().idxmax()
df_train['buildingqualitytypeid'] = df_train['buildingqualitytypeid'].fillna(buildingqual)

df_train.head(2)

,parcelid,logerror,transactiondate,airconditioningtypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,calculatedfinishedsquarefeet,finishedsquarefeet15,finishedsquarefeet50,fips,fireplacecnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,censustractandblock
0,14297519,0.025595,2017-01-01,5.0,0.0,3.5,4.0,8.0,3.5,0.0,3100.0,3100.0,3100.0,6059.0,0.0,2.0,633.0,0,13.0,33634932.0,-117869208.0,4506.0,0.0,0.0,0.0,0.0,122,261.0,NaN,60590632.0,53571.0,1286.0,96978.0,0.0,NaN,0.0,0.0,1998.0,1.0,0,485713.0,1023282.0,2016.0,537569.0,11013.719727,0,6.059063e+13
1,17052889,0.055619,2017-01-01,5.0,0.0,1.0,2.0,8.0,1.0,0.0,1465.0,1465.0,1465.0,6111.0,1.0,1.0,0.0,0,13.0,34449264.0,-119281528.0,12647.0,0.0,0.0,0.0,0.0,1110,261.0,NaN,61110012.0,13091.0,2061.0,97099.0,5.0,NaN,0.0,0.0,1967.0,1.0,1,88000.0,464000.0,2016.0,376000.0,5672.479980,0,6.111001e+13


- **"unitcnt"** - Number of units in a property. Change "NaN" to "1"

In [53]:
df_train.unitcnt.fillna(1,inplace = True)

- **"propertyzoningdesc"** - This seems like a very error-ridden column with so many unique values. It may provide some valuable info, so lets just fill the "NaN" with the most common value.

print(df_train['propertyzoningdesc'].value_counts())

In [54]:
# Fill in "NaN" "propertyzoningdesc" with most common
propertyzoningdesc = df_train['propertyzoningdesc'].value_counts().idxmax()
df_train['propertyzoningdesc'] = df_train['propertyzoningdesc'].fillna(propertyzoningdesc)

- **"lotsizesquarefeet"** - Area of lot in square feet. Fill "NaN" with average value between 25 to 75 tax value Quantile *REVIST THIS*

In [55]:
#lotsize_2575 = df_train.taxvaluedollarcnt.quartile(.25)

df_train['lotsizesquarefeet'].fillna((df_train['lotsizesquarefeet'].mean()), inplace=True)

- **"censustractandblock"** & **"rawcensustractandblock"** - Census tract and block ID combined. Look like duplicate values. I think we should drop these because they are related to location which is covered by "longitude" and "latitude". Let's view the values first.

In [56]:
# Drop "censustractandblock"
df_train.drop('censustractandblock', axis=1, inplace=True)

- **"landtaxvaluedollarcnt"** - Assessed value of land area of parcel.

- **"structuretaxvaluedollarcnt"** - Assessed value of built structure on land.

- **"taxvaluedollarcnt"** - Total tax assessed value of property. "structuretax..." + "landtax...".

- **"taxamount"** - Total property tax assessed for assessment year.

Let's filter our data and view the relationships of these columns. This should allow us to strategically fill in the blanks.

In [57]:
taxdata = df_train[df_train['landtaxvaluedollarcnt'].notnull() & 
                        df_train['structuretaxvaluedollarcnt'].notnull() & 
                        df_train['taxvaluedollarcnt'].notnull() & 
                        df_train['taxamount'].notnull()]
taxdata[['landtaxvaluedollarcnt','structuretaxvaluedollarcnt','taxvaluedollarcnt','taxamount']]

,landtaxvaluedollarcnt,structuretaxvaluedollarcnt,taxvaluedollarcnt,taxamount
0,537569.0,485713.0,1023282.0,11013.719727
1,376000.0,88000.0,464000.0,5672.479980
2,479489.0,85289.0,564778.0,6488.299805
3,36225.0,108918.0,145143.0,1777.510010
4,45726.0,73681.0,119407.0,1533.890015
5,132424.0,198640.0,331064.0,3508.100098
6,496619.0,276684.0,773303.0,9516.259766
7,199662.0,18890.0,218552.0,2366.080078
8,43056.0,177527.0,220583.0,3104.189941
9,220058.0,151303.0,371361.0,4557.319824


- **"landtaxvaluedollarcnt"** - We can fill in the "NaN"s with "0". It appears some properties do not have own any land. An example of this could be an apartment in large building where only the structurevalue would exist.

- **"structuretaxvaluedollarcnt"** - Same as **"landtaxvaluedollarcnt"**, but opposite. An example of a "NaN" in this category would be an empty lot.

- **"taxvaluedollarcnt"** - We can just fill in the "NaN" values with the average.

- **"taxamount"** - We should calculate a new category called *'taxpercentage'* where we divide the taxamount by the 'taxvaluedollarcnt', then we can fill in the "NaN" values with the average tax percentage.

In [58]:
df_train.landtaxvaluedollarcnt.fillna(0,inplace = True)

df_train.structuretaxvaluedollarcnt.fillna(0,inplace = True)

df_train['taxvaluedollarcnt'].fillna((df_train['taxvaluedollarcnt'].mean()), inplace=True)

In [59]:
df_train['taxpercentage'] = df_train['taxamount'] / df_train['taxvaluedollarcnt']
df_train.head(2)

,parcelid,logerror,transactiondate,airconditioningtypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,calculatedfinishedsquarefeet,finishedsquarefeet15,finishedsquarefeet50,fips,fireplacecnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxpercentage
0,14297519,0.025595,2017-01-01,5.0,0.0,3.5,4.0,8.0,3.5,0.0,3100.0,3100.0,3100.0,6059.0,0.0,2.0,633.0,0,13.0,33634932.0,-117869208.0,4506.0,0.0,0.0,0.0,0.0,122,261.0,LAR1,60590632.0,53571.0,1286.0,96978.0,0.0,1.0,0.0,0.0,1998.0,1.0,0,485713.0,1023282.0,2016.0,537569.0,11013.719727,0,0.010763
1,17052889,0.055619,2017-01-01,5.0,0.0,1.0,2.0,8.0,1.0,0.0,1465.0,1465.0,1465.0,6111.0,1.0,1.0,0.0,0,13.0,34449264.0,-119281528.0,12647.0,0.0,0.0,0.0,0.0,1110,261.0,LAR1,61110012.0,13091.0,2061.0,97099.0,5.0,1.0,0.0,0.0,1967.0,1.0,1,88000.0,464000.0,2016.0,376000.0,5672.479980,0,0.012225


In [60]:
df_train['taxpercentage'].fillna((df_train['taxpercentage'].mean()), inplace=True)

Now we will drop **"taxamount"** because we have replaced it with **"taxpercentage"**.

In [61]:
# Drop "taxamount"
df_train.drop('taxamount', axis=1, inplace=True)

- **"regionidcity"** - City property is located in. This is redundant information, so we will drop. *REVISIT THIS*

In [62]:
# Drop "regionidcity"
df_train.drop('regionidcity', axis=1, inplace=True)

- **"yearbuilt"** - Year home was built. We can just fill in the "NaN" values with the most common value.

In [63]:
# Fill in "NaN" "yearbuilt" with most common
yearbuilt = df_train['yearbuilt'].value_counts().idxmax()
df_train['yearbuilt'] = df_train['yearbuilt'].fillna(yearbuilt)

In [64]:
print(df_train.shape)
print(df_train.dtypes)
df_train.notnull().mean().sort_values(ascending = False)

(77613, 45)
parcelid                          int64
logerror                        float64
transactiondate                  object
airconditioningtypeid           float32
basementsqft                    float32
bathroomcnt                     float32
bedroomcnt                      float32
buildingqualitytypeid           float32
calculatedbathnbr               float32
decktypeid                      float32
calculatedfinishedsquarefeet    float32
finishedsquarefeet15            float32
finishedsquarefeet50            float32
fips                            float32
fireplacecnt                    float32
garagecarcnt                    float32
garagetotalsqft                 float32
hashottuborspa                    int64
heatingorsystemtypeid           float32
latitude                        float32
longitude                       float32
lotsizesquarefeet               float32
poolcnt                         float32
poolsizesum                     float32
pooltypeid2                 

taxpercentage                   1.000000
pooltypeid2                     1.000000
taxdelinquencyflag              1.000000
lotsizesquarefeet               1.000000
heatingorsystemtypeid           1.000000
hashottuborspa                  1.000000
garagetotalsqft                 1.000000
garagecarcnt                    1.000000
fireplacecnt                    1.000000
finishedsquarefeet50            1.000000
finishedsquarefeet15            1.000000
calculatedfinishedsquarefeet    1.000000
decktypeid                      1.000000
calculatedbathnbr               1.000000
buildingqualitytypeid           1.000000
basementsqft                    1.000000
airconditioningtypeid           1.000000
transactiondate                 1.000000
logerror                        1.000000
poolsizesum                     1.000000
poolcnt                         1.000000
pooltypeid7                     1.000000
unitcnt                         1.000000
landtaxvaluedollarcnt           1.000000
taxvaluedollarcn

In [65]:
# Fill in "fips" "NaN"s
fips = df_train['fips'].value_counts().idxmax()
df_train['fips'] = df_train['fips'].fillna(fips)

# Fill in "propertylandusetypeid" "NaN"s
propertylandusetypeid = df_train['propertylandusetypeid'].value_counts().idxmax()
df_train['propertylandusetypeid'] = df_train['propertylandusetypeid'].fillna(propertylandusetypeid)

# Drop 'regionidcounty' - REVIST LATER
# df_train.drop('regionidcounty', axis=1, inplace=True)

# Fill in "latitude" "NaN"s
latitude = df_train['latitude'].value_counts().idxmax()
df_train['latitude'] = df_train['latitude'].fillna(latitude)

# Fill in "longitude" "NaN"s
longitude = df_train['longitude'].value_counts().idxmax()
df_train['longitude'] = df_train['longitude'].fillna(longitude)

# Fill in "rawcensustractandblock" "NaN"s
rawcensustractandblock = df_train['rawcensustractandblock'].value_counts().idxmax()
df_train['rawcensustractandblock'] = df_train['rawcensustractandblock'].fillna(rawcensustractandblock)

# Fill in "assessmentyear" "NaN"s
assessmentyear = df_train['assessmentyear'].value_counts().idxmax()
df_train['assessmentyear'] = df_train['assessmentyear'].fillna(assessmentyear)

# Fill in "bedroomcnt" "NaN"s
bedroomcnt = df_train['bedroomcnt'].value_counts().idxmax()
df_train['bedroomcnt'] = df_train['bedroomcnt'].fillna(bedroomcnt)

# Fill in "bathroomcnt" "NaN"s
bathroomcnt = df_train['bathroomcnt'].value_counts().idxmax()
df_train['bathroomcnt'] = df_train['bathroomcnt'].fillna(bathroomcnt)

# Fill in "roomcnt" "NaN"s
roomcnt = df_train['roomcnt'].value_counts().idxmax()
df_train['roomcnt'] = df_train['roomcnt'].fillna(roomcnt)

# Fill in "propertycountylandusecode" "NaN"s
propertycountylandusecode = df_train['propertycountylandusecode'].value_counts().idxmax()
df_train['propertycountylandusecode'] = df_train['propertycountylandusecode'].fillna(propertycountylandusecode)

# Fill in "regionidzip " "NaN"s
regionidzip = df_train['regionidzip'].value_counts().idxmax()
df_train['regionidzip'] = df_train['regionidzip'].fillna(regionidzip)

In [67]:
df_train.columns

Index(['parcelid', 'logerror', 'transactiondate', 'airconditioningtypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid',
       'calculatedbathnbr', 'decktypeid', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet15', 'finishedsquarefeet50', 'fips', 'fireplacecnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcounty',
       'regionidzip', 'roomcnt', 'unitcnt', 'yardbuildingsqft17',
       'yardbuildingsqft26', 'yearbuilt', 'numberofstories', 'fireplaceflag',
       'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear',
       'landtaxvaluedollarcnt', 'taxdelinquencyflag', 'taxpercentage'],
      dtype='object')

In [68]:
# check missing values
df_train[df_train.isnull().any(axis=1)]

,parcelid,logerror,transactiondate,airconditioningtypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,calculatedfinishedsquarefeet,finishedsquarefeet15,finishedsquarefeet50,fips,fireplacecnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcounty,regionidzip,roomcnt,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxdelinquencyflag,taxpercentage
388,10813418,-0.008083,2017-01-03,5.0,0.0,2.0,3.0,8.0,2.0,0.0,1784.939819,1784.939819,1784.939819,6037.0,0.0,0.0,0.0,0,13.0,33984600.0,-118435000.0,29973.839844,0.0,0.0,0.0,0.0,0100,261.0,LAR1,60590320.0,NaN,97319.0,0.0,1.0,0.0,0.0,1955.0,1.0,0,0.0,490151.5625,2016.0,0.0,0,0.01317
1083,12133448,-1.062436,2017-01-05,5.0,0.0,2.0,3.0,8.0,2.0,0.0,1784.939819,1784.939819,1784.939819,6037.0,0.0,0.0,0.0,0,13.0,33984600.0,-118435000.0,29973.839844,0.0,0.0,0.0,0.0,0100,261.0,LAR1,60590320.0,NaN,97319.0,0.0,1.0,0.0,0.0,1955.0,1.0,0,0.0,490151.5625,2016.0,0.0,0,0.01317
3576,11905748,0.084014,2017-01-18,5.0,0.0,2.0,3.0,8.0,2.0,0.0,1784.939819,1784.939819,1784.939819,6037.0,0.0,0.0,0.0,0,13.0,33984600.0,-118435000.0,29973.839844,0.0,0.0,0.0,0.0,0100,261.0,LAR1,60590320.0,NaN,97319.0,0.0,1.0,0.0,0.0,1955.0,1.0,0,0.0,490151.5625,2016.0,0.0,0,0.01317
3726,12961343,0.559867,2017-01-18,5.0,0.0,2.0,3.0,8.0,2.0,0.0,1784.939819,1784.939819,1784.939819,6037.0,0.0,0.0,0.0,0,13.0,33984600.0,-118435000.0,29973.839844,0.0,0.0,0.0,0.0,0100,261.0,LAR1,60590320.0,NaN,97319.0,0.0,1.0,0.0,0.0,1955.0,1.0,0,0.0,490151.5625,2016.0,0.0,0,0.01317
3782,12006414,-0.795601,2017-01-18,5.0,0.0,2.0,3.0,8.0,2.0,0.0,1784.939819,1784.939819,1784.939819,6037.0,0.0,0.0,0.0,0,13.0,33984600.0,-118435000.0,29973.839844,0.0,0.0,0.0,0.0,0100,261.0,LAR1,60590320.0,NaN,97319.0,0.0,1.0,0.0,0.0,1955.0,1.0,0,0.0,490151.5625,2016.0,0.0,0,0.01317
4913,12039176,0.175751,2017-01-24,5.0,0.0,2.0,3.0,8.0,2.0,0.0,1784.939819,1784.939819,1784.939819,6037.0,0.0,0.0,0.0,0,13.0,33984600.0,-118435000.0,29973.839844,0.0,0.0,0.0,0.0,0100,261.0,LAR1,60590320.0,NaN,97319.0,0.0,1.0,0.0,0.0,1955.0,1.0,0,0.0,490151.5625,2016.0,0.0,0,0.01317
8888,11905738,-0.050252,2017-02-09,5.0,0.0,2.0,3.0,8.0,2.0,0.0,1784.939819,1784.939819,1784.939819,6037.0,0.0,0.0,0.0,0,13.0,33984600.0,-118435000.0,29973.839844,0.0,0.0,0.0,0.0,0100,261.0,LAR1,60590320.0,NaN,97319.0,0.0,1.0,0.0,0.0,1955.0,1.0,0,0.0,490151.5625,2016.0,0.0,0,0.01317
9323,12185469,-0.447673,2017-02-10,5.0,0.0,2.0,3.0,8.0,2.0,0.0,1784.939819,1784.939819,1784.939819,6037.0,0.0,0.0,0.0,0,13.0,33984600.0,-118435000.0,29973.839844,0.0,0.0,0.0,0.0,0100,261.0,LAR1,60590320.0,NaN,97319.0,0.0,1.0,0.0,0.0,1955.0,1.0,0,0.0,490151.5625,2016.0,0.0,0,0.01317
10467,11905737,-0.029747,2017-02-16,5.0,0.0,2.0,3.0,8.0,2.0,0.0,1784.939819,1784.939819,1784.939819,6037.0,0.0,0.0,0.0,0,13.0,33984600.0,-118435000.0,29973.839844,0.0,0.0,0.0,0.0,0100,261.0,LAR1,60590320.0,NaN,97319.0,0.0,1.0,0.0,0.0,1955.0,1.0,0,0.0,490151.5625,2016.0,0.0,0,0.01317
11427,10855501,0.018408,2017-02-22,5.0,0.0,2.0,3.0,8.0,2.0,0.0,1784.939819,1784.939819,1784.939819,6037.0,0.0,0.0,0.0,0,13.0,33984600.0,-118435000.0,29973.839844,0.0,0.0,0.0,0.0,0100,261.0,LAR1,60590320.0,NaN,97319.0,0.0,1.0,0.0,0.0,1955.0,1.0,0,0.0,490151.5625,2016.0,0.0,0,0.01317


In [69]:
# # There is a missing value in regionidcounty. The houses with same regionidzip should have same regionidcounty

df_train.loc[(df_train['regionidcounty'].isnull())] = 97319

In [70]:
#drop parcelid
df_train= df_train.drop('parcelid',axis=1)

In [71]:
# check missing values
df_train[df_train.isnull().any(axis=1)]

,logerror,transactiondate,airconditioningtypeid,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,calculatedfinishedsquarefeet,finishedsquarefeet15,finishedsquarefeet50,fips,fireplacecnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcounty,regionidzip,roomcnt,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxdelinquencyflag,taxpercentage


In [72]:
# Display categorical and numerical variables

numeric_var = [key for key in dict(df_train.dtypes)
                   if dict(df_train.dtypes)[key]
                       in ['float64','float32','int32','int64']] # Numeric Variable

cat_var = [key for key in dict(df_train.dtypes)
             if dict(df_train.dtypes)[key] in ['object'] ] # Categorical Varible

In [73]:
cat_var

['transactiondate', 'propertycountylandusecode', 'propertyzoningdesc']

In [74]:
numeric_var

['logerror',
 'airconditioningtypeid',
 'basementsqft',
 'bathroomcnt',
 'bedroomcnt',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'decktypeid',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'fips',
 'fireplacecnt',
 'garagecarcnt',
 'garagetotalsqft',
 'hashottuborspa',
 'heatingorsystemtypeid',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'poolcnt',
 'poolsizesum',
 'pooltypeid2',
 'pooltypeid7',
 'propertylandusetypeid',
 'rawcensustractandblock',
 'regionidcounty',
 'regionidzip',
 'roomcnt',
 'unitcnt',
 'yardbuildingsqft17',
 'yardbuildingsqft26',
 'yearbuilt',
 'numberofstories',
 'fireplaceflag',
 'structuretaxvaluedollarcnt',
 'taxvaluedollarcnt',
 'assessmentyear',
 'landtaxvaluedollarcnt',
 'taxdelinquencyflag',
 'taxpercentage']

In [75]:
# convert transactiondate to transaction month and then transaction quarter
df_train['transaction_mon'] = pd.DatetimeIndex(df_train['transactiondate']).month
    
df_train.transaction_mon[df_train.transaction_mon == 2] = 1   
df_train.transaction_mon[df_train.transaction_mon == 3] = 1 
df_train.transaction_mon[df_train.transaction_mon == 4] = 2
df_train.transaction_mon[df_train.transaction_mon == 5] = 2
df_train.transaction_mon[df_train.transaction_mon == 6] = 2
df_train.transaction_mon[df_train.transaction_mon == 7] = 3
df_train.transaction_mon[df_train.transaction_mon == 8] = 3
df_train.transaction_mon[df_train.transaction_mon == 9] = 3
df_train.transaction_mon[df_train.transaction_mon == 10] = 4
df_train.transaction_mon[df_train.transaction_mon == 11] = 4
df_train.transaction_mon[df_train.transaction_mon == 12] = 4

In [76]:
df_train.rename(columns={'transaction_mon': 'transaction_quarter'}, inplace=True)

# dropping propertyzoningdesc and propertycountylandusecode because of field redundancy
df_train= df_train.drop('propertyzoningdesc',axis=1)
df_train= df_train.drop('propertycountylandusecode',axis=1)



In [77]:
df_train= df_train.drop('transactiondate',axis=1)

In [78]:
df_train.shape

(77613, 42)

In [ ]:
df_train.to_csv("C:/Users/Deepika Keswarap/Documents/projecta/all/Zillowdata2017.csv", index=False, encoding='utf8')